In [11]:
import tensorflow as tf
import numpy as np
from PIL import Image

In [13]:
# import mnist data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [138]:
# dataset size
n_train = mnist.train.num_examples # 55,000
n_validation = mnist.validation.num_examples # 5000
n_test = mnist.test.num_examples # 10,000

In [139]:
# network architecture
n_input = 784 	# input layer (28x28 pixels)
n_hidden1 = 512 # 1st hidden layer
n_hidden2 = 256 # 2nd hidden layer
n_hidden3 = 128 # 3rd hidden layer
n_output = 10 	# output layer (0-9 digits)

In [140]:
# network parameters
learning_rate = 1e-4
n_iterations = 1000
batch_size = 128
dropout = 0.5

In [141]:
# tf placeholders
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
keep_prob = tf.placeholder(tf.float32)

In [142]:
# w & b parameters
weights = {
	'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
	'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
	'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
	'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
}
biases = {
	'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
	'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
	'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
	'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

In [143]:
# network layers
layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.matmul(layer_3, weights['out']) + biases['out']

In [144]:
# define loss and optimiser
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output_layer))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [145]:
# define evaluation
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [146]:
# initialise variables, start session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [147]:
# train on minibatches
for i in range(n_iterations):
	batch_x, batch_y = mnist.train.next_batch(batch_size)
	sess.run(train_step, feed_dict={X: batch_x, Y: batch_y, keep_prob:dropout})
	
	# print loss and accuracy (per minibatch)
	if i%100==0:
		minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batch_x, Y: batch_y, keep_prob:1.0})
		print("Iteration", str(i), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))


Iteration 0 	| Loss = 3.59507 	| Accuracy = 0.09375
Iteration 100 	| Loss = 0.539291 	| Accuracy = 0.859375
Iteration 200 	| Loss = 0.473393 	| Accuracy = 0.828125
Iteration 300 	| Loss = 0.384827 	| Accuracy = 0.890625
Iteration 400 	| Loss = 0.300069 	| Accuracy = 0.90625
Iteration 500 	| Loss = 0.392261 	| Accuracy = 0.890625
Iteration 600 	| Loss = 0.228807 	| Accuracy = 0.953125
Iteration 700 	| Loss = 0.406879 	| Accuracy = 0.898438
Iteration 800 	| Loss = 0.213302 	| Accuracy = 0.929688
Iteration 900 	| Loss = 0.237281 	| Accuracy = 0.914062


In [148]:
# accuracy on test set
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0})
print("\nAccuracy on test set:", test_accuracy)


Accuracy on test set: 0.9182


In [149]:
test_img=Image.open("test_img.png")
test_img

In [150]:
# accuracy on own image
img = np.invert(Image.open("test_img.png").convert('L')).ravel()
prediction = sess.run(tf.argmax(output_layer,1), feed_dict={X: [img]})
print ("Prediction for test image:", np.squeeze(prediction))

Prediction for test image: 2


In [151]:
im = Image.open("image_3.png")
im_resized = im.resize(size, Image.ANTIALIAS)
im_resized.save("image_3_resized.png", "PNG")

In [152]:
img_3 = Image.open("image_3_resized.png")
img_name="image_3_resized.png"
img_3

In [153]:
img_3_digit = np.invert(Image.open("image_3_resized.png").convert('L')).ravel()
prediction = sess.run(tf.argmax(output_layer,1), feed_dict={X: [img_3_digit]})
print ("Prediction for {img}:".format(img=img_name), np.squeeze(prediction))

Prediction for image_3_resized.png: 3


In [154]:
size = 28, 28
im = Image.open("image_8_test.png")
im_resized = im.resize(size, Image.ANTIALIAS)
im_resized.save("image_8_resized.png", "PNG")

In [155]:
img_8 = Image.open("image_8_resized.png")
img_name="image_8_resized.png"
img_8

In [156]:
img_8_digit = np.invert(Image.open("image_8_resized.png").convert('L')).ravel()
prediction = sess.run(tf.argmax(output_layer,1), feed_dict={X: [img_8_digit]})
print ("Prediction for {img}:".format(img=img_name), np.squeeze(prediction))

Prediction for image_8_resized.png: 8
